In [1]:
import numpy as np


while True:
  probabilidad_letra = {'Letra 0': [' ', 'e', 'a', 'o', 'i', 'n', 's', 'd', 'r', 'u', 'c', 'l', 't', 'ñ', 'p',
                                  'm', 'b', 'g', 'v', 'f', 'y', 'j', 'q', 'h', 'z', 'x', 'w', 'k'],
                        'P0': None}

  # Generar una lista de 27 números aleatorios entre 0 y 1
  random_numbers = np.random.rand(len(probabilidad_letra['Letra 0']))

  # Normalizar la lista para que la suma sea igual a 1
  normalized_numbers = random_numbers / sum(random_numbers)

  # Organiar la lista de mayor a menor
  normalized_numbers_sorted = sorted(normalized_numbers, reverse=True)

  # Agregar la list organizada de probabilidades al diccionario "probabilidad_letra"
  probabilidad_letra['P0'] = normalized_numbers_sorted


  import pandas as pd

  pd.set_option('display.max_columns', None)

  # Crear el DataFrame e función del ciccionario "probabilidad_letra"
  df = pd.DataFrame(probabilidad_letra)
  df_copy = df.copy()

  # Lista con los número del 0 al 27 (No incluye el 27)
  # La longitud de la lista será las veces que se repetira el siguiente for
  bits = list(range(df_copy['P0'].count()-1))

  for i_bit in range(len(bits)):
    # Asignar los bits 0 y 1 a las últimas posiciones de cada columna en cada iteración del "for"
    df_copy[f'Bit {i_bit}'] = df_copy['P0'].apply(lambda x: 1 if df_copy['P0'].iloc[-(i_bit+1)]==x
                                                            else (0 if df_copy['P0'].iloc[-(i_bit+2)]==x else '-'))

    # Sumar las posiciones donde se encuentran los bits 0 y 1
    suma_ultimas_2_posciones = df_copy[f'P{i_bit}'][(df_copy[f'Bit {i_bit}']==1) | (df_copy[f'Bit {i_bit}']==0)].sum()

    # Crear las nuevas probabilidades
    if i_bit < len(bits)-1:

      # Crear la columna "Letra ..." con valores "None" en todas sus filas
      df_copy[f'Letra {i_bit+1}'] = None

      '''
      FUNCIÓN para asignar las nuevas probabilidades de cada letra

      Argumetnos:
        x: Probabilidad de cada fila de la columna "P..." directamente anterior a la actual,
        flag_sum: Validadr el acceso a la condicción de "(x > suma_ultimas_2_posciones)",
        suma_ultimas_2_posciones: Suma de las probabilidad donde se ubican el
        bit "0" y bit "1" de la columna "P..." directamente anterior a la actual
      '''
      # Variable global para gestionar el acceso únicamente por una sola vez al "elif"
      se_cumplio_elif = False

      def asignar_probabilidad(x, flag_sum, suma_ultimas_2_posciones):
          global se_cumplio_elif

          # Obtener el índice de la probabilidad de la columna "P..." directamente anterior a la actual
          indice_letra = df_copy[f'Letra {i_bit}'][df_copy[f'P{i_bit}']==x].index[0]

          # Si se cumple la condición de: (x > suma_ultimas_2_posciones)
          if flag_sum:
            # Restablecer la variable de estado cuando se cumpla la condición if
            se_cumplio_elif = False

            # Actualizar el valor de la columna recien creada "df_copy[f'Letra {i_bit+1}'] = None"
            # en el índice encontrado y almacenado en "indice_letra"
            df_copy.at[indice_letra, f'Letra {i_bit+1}'] = df_copy[f'Letra {i_bit}'].iloc[indice_letra]

            # Retorno que será almacenado en "df_copy[f'P{i_bit+1}']"
            return x

          # Este "elif" solo se ejecutará una sola vez, cuando no se cumple por primera vez
          # la condición de: "(x > suma_ultimas_2_posciones)"
          elif not se_cumplio_elif:
            # Cumplir la condición elif solo si no se ha cumplido antes
            se_cumplio_elif = True

            indice_letra_sum_igual = df_copy[f'Bit {i_bit}'][df_copy[f'Bit {i_bit}']==0].index[0]
            df_copy.at[indice_letra, f'Letra {i_bit+1}'] = df_copy[f'Letra {i_bit}'].iloc[indice_letra_sum_igual] + \
                                                                      df_copy[f'Letra {i_bit}'].iloc[indice_letra_sum_igual+1]

            # Retorno que será almacenado en "df_copy[f'P{i_bit+1}']"
            return suma_ultimas_2_posciones

          # El "else" se siempre y cuando no se halla cumplido por primera vez
          # la condición de: "(x > suma_ultimas_2_posciones)" y se cumplira mientras
          # queden filas por aginar
          else:

            # Actualizar el valor de la columna recien creada "df_copy[f'Letra {i_bit+1}'] = None"
            # en el índice encontrado y almacenado en "indice_letra"
            df_copy.at[indice_letra, f'Letra {i_bit+1}'] = df_copy[f'Letra {i_bit}'].iloc[indice_letra-1]

            # Retorno que será almacenado en "df_copy[f'P{i_bit+1}']"
            return 0
      '''
      Fin de la FUNCIÓN: "asignar_probabilidad()"
      '''

      # Crear nueva columna "df_copy[f'P...']"
      df_copy[f'P{i_bit+1}'] = df_copy[f'P{i_bit}'].apply(lambda x: asignar_probabilidad(x, True, 0) if x>suma_ultimas_2_posciones
                                                          else asignar_probabilidad(x, False, suma_ultimas_2_posciones))

      # Crear arreglo con las probabilidades que son menores a la suma de los bits "0" y "1"
      pos_arr = [j for j in df_copy[f'P{i_bit}'] if j < suma_ultimas_2_posciones]

      # Encontrar los índices de las probabilidades que son menores al valor de la suma de los bits 0 y 1
      # en función de la longitu de la lista obtenida y almacena en "pos_arr"
      rows_no_sum = df_copy[f'P{i_bit}'].iloc[(len(pos_arr)-1)*(-1):(i_bit+1)*(-1)]
      indices_no_sum = rows_no_sum.index.tolist()

      # Iterar sobre la lista de indices encontrada (tipo int)
      # Para actualizar las probabilidades cruzadas
      for i_posarr in indices_no_sum:
        indice_anterior = i_posarr - 1
        valor_anterior = df_copy[f'P{i_bit}'].iloc[indice_anterior]
        df_copy.at[i_posarr, f'P{i_bit+1}'] = valor_anterior

      # La filas de la columna df_copy[f'P...'] que no conttrandan nigun valor se les asigna un valor núemrico "np.NaN"
      indices_valor_cero = df_copy[f'P{i_bit+1}'].loc[df_copy[f'P{i_bit+1}'] == 0].index
      for i_reset_p_0 in indices_valor_cero:
          df_copy.loc[i_reset_p_0, f'Letra {i_bit+1}'] = np.NaN

  '''
  Iniciar  con el cálculo de las codificación de cada letra
  '''
  import re

  # Se almacenarán las letras y su respectiva codificación calculada
  dict_lettters = {}

  # Adjuntar las los caracteres (espacio y letras) al nuevo diccionario "dict_lettters"
  for i in probabilidad_letra['Letra 0']:
    dict_lettters[i] = ''

  # Obtener una lista con los nombre de todas las columnas de "df_copy" de tipo "Bit ..."
  # usando expresiones regulares
  list_columns_bits = [i for i in df_copy.columns if re.match(r'Bit \d+', i)]
  # Daarle la vuelta a la lista "list_columns_bits" para encontrar el bit de mayor peso de primeras
  list_columns_bits.reverse()

  # Caluclar las codificaciones de cada letra
  temp = len(list_columns_bits)-1
  for i_bits_columns in list_columns_bits:
    # Obtener el string que contiene las letras que están en los bit 0 y bit 1
    bit_0_y_1 = [(df_copy[f'{i_bits_columns}'][(df_copy[f'{i_bits_columns}']==0)].index[0],
                df_copy[f'{i_bits_columns}'][(df_copy[f'{i_bits_columns}']==1)].index[0])]

    # Iterar sobre el string (que puede contener solo una letra o varias letras) de cada uno de los bits (0 y 1)
    for i_bit_0, i_bit_1 in bit_0_y_1:
      # Obtener letras con el bit 0
      for i_0 in df_copy[f'Letra {temp}'].iloc[i_bit_0]:
        dict_lettters[i_0] += '0'

      # Obtener letras con el bit 1
      for j in df_copy[f'Letra {temp}'].iloc[i_bit_1]:
        dict_lettters[j] += '1'

      #
      df_copy[f'Letra {temp}'].iloc[i_bit_1]

    temp-=1

  df_codificacion = pd.DataFrame(list(dict_lettters.items()), columns=['Letra', 'Codificación'])
  # df_codificacion["Fill 0s"] = df_codificacion['Codificación'].apply(lambda x: x.zfill(len(df_codificacion['Codificación'].iloc[-1])))
  df_codificacion["Fill 0s"] = df_codificacion['Codificación'].apply(lambda x: x.zfill(8))
  df_codificacion['Longitud'] = df_codificacion["Fill 0s"].apply(lambda x: len(x))

  if len(df_codificacion['Codificación'].iloc[-1]) == 8:
    break

In [2]:
print("Dataframe incial:\n")
df

Dataframe incial:



,Letra 0,P0
0,,0.066091
1,e,0.064545
2,a,0.063925
3,o,0.062129
4,i,0.059159
5,n,0.053763
6,s,0.052092
7,d,0.051845
8,r,0.050561
9,u,0.049606


In [3]:
print("Dataframe con las diferentes probabilidades de las configuraciones de letras calculada:\n")
df_copy

Dataframe con las diferentes probabilidades de las configuraciones de letras calculada:



,Letra 0,P0,Bit 0,Letra 1,P1,Bit 1,Letra 2,P2,Bit 2,Letra 3,P3,Bit 3,Letra 4,P4,Bit 4,Letra 5,P5,Bit 5,Letra 6,P6,Bit 6,Letra 7,P7,Bit 7,Letra 8,P8,Bit 8,Letra 9,P9,Bit 9,Letra 10,P10,Bit 10,Letra 11,P11,Bit 11,Letra 12,P12,Bit 12,Letra 13,P13,Bit 13,Letra 14,P14,Bit 14,Letra 15,P15,Bit 15,Letra 16,P16,Bit 16,Letra 17,P17,Bit 17,Letra 18,P18,Bit 18,Letra 19,P19,Bit 19,Letra 20,P20,Bit 20,Letra 21,P21,Bit 21,Letra 22,P22,Bit 22,Letra 23,P23,Bit 23,Letra 24,P24,Bit 24,Letra 25,P25,Bit 25,Letra 26,P26,Bit 26
0,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,-,njqv,0.106693,-,ibg,0.117533,-,ao,0.126053,-,yzxme,0.129467,-,hwkfp,0.143634,-,cltñ,0.172515,-,sdru,0.204105,-,ibgnjqv,0.224226,-,yzxmeao,0.255520,-,cltñhwkfp,0.316149,-,ibgnjqvsdru,0.428331,-,cltñhwkfp yzxmeao,0.571669,0
1,e,0.064545,-,e,0.064545,-,e,0.064545,-,e,0.064545,-,e,0.064545,-,e,0.064545,-,e,0.064545,-,e,0.064545,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,-,njqv,0.106693,-,ibg,0.117533,-,ao,0.126053,-,yzxme,0.129467,-,hwkfp,0.143634,-,cltñ,0.172515,-,sdru,0.204105,-,ibgnjqv,0.224226,-,yzxmeao,0.255520,-,cltñhwkfp,0.316149,0,ibgnjqvsdru,0.428331,1
2,a,0.063925,-,a,0.063925,-,a,0.063925,-,a,0.063925,-,a,0.063925,-,a,0.063925,-,a,0.063925,-,a,0.063925,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,-,njqv,0.106693,-,ibg,0.117533,-,ao,0.126053,-,yzxme,0.129467,-,hwkfp,0.143634,-,cltñ,0.172515,-,sdru,0.204105,-,ibgnjqv,0.224226,0,yzxmeao,0.255520,1,NaN,0.000000,-
3,o,0.062129,-,o,0.062129,-,o,0.062129,-,o,0.062129,-,o,0.062129,-,o,0.062129,-,o,0.062129,-,o,0.062129,-,o,0.062129,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,-,njqv,0.106693,-,ibg,0.117533,-,ao,0.126053,-,yzxme,0.129467,-,hwkfp,0.143634,-,cltñ,0.172515,0,sdru,0.204105,1,NaN,0.000000,-,NaN,0.000000,-
4,i,0.059159,-,i,0.059159,-,i,0.059159,-,i,0.059159,-,i,0.059159,-,i,0.059159,-,i,0.059159,-,i,0.059159,-,i,0.059159,-,o,0.062129,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,-,njqv,0.106693,-,ibg,0.117533,-,ao,0.126053,-,yzxme,0.129467,0,hwkfp,0.143634,1,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-
5,n,0.053763,-,n,0.053763,-,n,0.053763,-,n,0.053763,-,n,0.053763,-,n,0.053763,-,n,0.053763,-,n,0.053763,-,bg,0.058374,-,i,0.059159,-,o,0.062129,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,-,njqv,0.106693,-,ibg,0.117533,0,ao,0.126053,1,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-
6,s,0.052092,-,s,0.052092,-,s,0.052092,-,s,0.052092,-,s,0.052092,-,s,0.052092,-,s,0.052092,-,jqv,0.052930,-,n,0.053763,-,bg,0.058374,-,i,0.059159,-,o,0.062129,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,-,ru,0.100168,-,sd,0.103938,0,njqv,0.106693,1,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-
7,d,0.051845,-,d,0.051845,-,d,0.051845,-,d,0.051845,-,d,0.051845,-,d,0.051845,-,d,0.051845,-,s,0.052092,-,jqv,0.052930,-,n,0.053763,-,bg,0.058374,-,i,0.059159,-,o,0.062129,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,-,tñ,0.081473,-,cl,0.091042,0,ru,0.100168,1,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-
8,r,0.050561,-,r,0.050561,-,r,0.050561,-,r,0.050561,-,r,0.050561,-,r,0.050561,-,r,0.050561,-,d,0.051845,-,s,0.052092,-,jqv,0.052930,-,n,0.053763,-,bg,0.058374,-,i,0.059159,-,o,0.062129,-,a,0.063925,-,e,0.064545,-,yzxm,0.064922,-,,0.066091,-,hwkfp,0.077543,0,tñ,0.081473,1,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.000000,-,NaN,0.00000

In [4]:
print('\nDataframe con la codificación de las letras:\n')
df_codificacion[['Letra',	'Codificación']]


Dataframe con la codificación de las letras:



,Letra,Codificación
0,,0011
1,e,0101
2,a,0110
3,o,0111
4,i,1000
5,n,1010
6,s,1100
7,d,1101
8,r,1110
9,u,1111


In [5]:
# Ingresar palabra o frase a codificar

while True:
  codificacion_palabra = ''

  try:
    palabra = input('\nIngrese la palabra o frase a codificar por Huffman:\n_> ').lower()

    for i_palabra in palabra:
      if i_palabra not in list(df_codificacion.Letra):
        raise(print(f'¡ERROR!: la palabra ingresada tiene el caracterer no permitido < {i_palabra} >. Intentelo de nuevo.'))

      else:
        codificacion_palabra += '{}'.format(df_codificacion['Fill 0s'][df_codificacion['Letra']==i_palabra].values[0])

    break

  except:
    continue

print('\nPalabra:', palabra)
print('Codificacion de la palabra:', codificacion_palabra)
print('Longitud de la palabra:', len(codificacion_palabra))



Ingrese la palabra o frase a codificar por Huffman:
_> hola mundo

Palabra: hola mundo
Codificacion de la palabra: 00010000000001110000000100000110000000110000100100001111000010100000110100000111
Longitud de la palabra: 80


In [ ]:
# Ingresar el bit de error sobre la palabra codificada
codificacion_palabra_error = '00010000000001110000000100000110000000110000100100001111000010100000110100000111'

In [ ]:
def calcular_paridad(palabra):
  # Convertir el str de la palabra codificada en una lista
  list_codificiacion = [i for i in palabra]

  # Agregar las posiciones de paridad en la lista de la palabra codificada
  list_codificiacion.insert(0, 'P')
  list_codificiacion.insert(1, 'P')
  list_codificiacion.insert(3, 'P')
  list_codificiacion.insert(7, 'P')

  '''
  Diccionario con las posiciones binarias de cada posción de la palabra codificada
  a la que se le agregó las posiciones de las paridades.
  Las posiciones binarias inician desde la posción 1 hasta la longitud total
  de la palbra codificada a la que se le agregó las posiciones de las paridades.
  '''
  dict_palabra = {'Palabra + Paridad': list_codificiacion,
                'Posición Binaria': []}

  # Función para convertir decimal a binario
  def decimal_a_binario(decimal):
      return bin(decimal)

  # Agregar las posicones binarias al diccionario "dict_palabra"
  for i_bin in range(1, 13):
    binario = decimal_a_binario(i_bin)
    # list_binario.append(binario[2:])
    dict_palabra['Posición Binaria'].append(binario[2:])

  # Calcular la longitud más larga entre las posicones binarias generadas
  len_max_posicion_binaria = len(dict_palabra['Posición Binaria'][-1])

  # Ajustar la longitud igual en todos los números binarios generados
  dict_palabra['Posición Binaria Ajustada'] = [i.zfill(len_max_posicion_binaria) for i in dict_palabra['Posición Binaria']]

  # Calcular la longitud de caracteres en las posiciones generadas binarias ajustadas
  # Recordar que las posiciones generadas binarias ajustadas tiene la misma longitud
  len_posicion_binaria = len(max(dict_palabra['Posición Binaria Ajustada']))

  '''
  Calcular las paridades de la palabra codificada
  '''
  # Para tabla de detección de errores
  list_paridades = []

  # Iterar sobre la longitud de las posiciones binarias
  for i_paridad in range(-1, -(len_posicion_binaria+1), -1):
    # Iterar sobre la palabra codificada a la que se le agregó las posiciones
    # de las paridades, y cada una de las posiciones binarias ajustadas
    sum_paridad = 0
    for i_paridad_palabra, j_num_bin in tuple(zip(dict_palabra['Palabra + Paridad'], dict_palabra['Posición Binaria Ajustada'])):
      # Verificar que la posición binaria ajustada no sea una posición de paridad
      if (i_paridad_palabra != 'P') and (j_num_bin[i_paridad] =='1'):
        # Si la posicón de la palabra es igual a 1
        if i_paridad_palabra == '1':
          sum_paridad += 1

    # Valor que se almacena en "list_paridades"
    temp_paridad = 0

    # Buscar el índice de la posicón de la paridad a actualizar su valor
    indice_paridad_act = dict_palabra['Palabra + Paridad'].index('P')

    '''
    Calcular el valor de la paridad
    0: si la suma par.
    1: si la suma impar.
    '''
    if sum_paridad % 2 == 0:
      temp_paridad = '0'
    else:
      temp_paridad = '1'

    # Actualizar el valor de la paridad en la palabra
    dict_palabra['Palabra + Paridad'][indice_paridad_act] = f'{temp_paridad}'

    # Almacenar el valor de la paridad ara tabla de detección de errores
    list_paridades.append(temp_paridad)

  return dict_palabra['Palabra + Paridad'], list_paridades

In [ ]:
# Caluclar paridades de un solo caracter (8 bits)
temp_letra_inferior = 0
temp_letra_superior = 8

for i in range(0, len(codificacion_palabra), 8):
  # Calcular la paridad orginal
  palabra_original_rx, paridad_original = calcular_paridad(codificacion_palabra[temp_letra_inferior:temp_letra_superior])

  # Calcular la paridad calculada
  palabra_calculada_rx, paridad_calculada = calcular_paridad(codificacion_palabra_error[temp_letra_inferior:temp_letra_superior])

  # print('\n')
  # print('═'*100)
  # print('palabra_original_rx:', palabra_original_rx, '\npalabra_calculada_rx:', palabra_calculada_rx)
  # print('paridad_original', paridad_original, '\nparidad_calculada', paridad_calculada)

  # Verificar Error
  # Empaquetar las paridades halladas (original y calculada)
  paridades_calculadas = tuple(zip(paridad_original, paridad_calculada))

  # Almacena la posición del posible error
  list_error = []

  # desempaquetar la tupla de pardidades para calcular si las paridades
  # (original y calculada) son iguales o no en cada posición
  for i, j in paridades_calculadas:
    if i==j:
      list_error.append('0')
    else:
      list_error.append('1')

  # Organizar
  list_error.reverse()

  # Conertir la lista en string de binario
  posicion_error_binario = ''.join(list_error)

  # Pasar la posción del error de binario a decimal
  indice_error = int(posicion_error_binario, 2)-1

  if indice_error >= 0:
    print('\n', '═'*100)
    print('Hay un error en la siguiente letra:\n')
    print('Palabra original:', palabra_original_rx, '\nPalabra calculada:', palabra_calculada_rx)
    print('Paridad original', paridad_original, '\nParidad calculada', paridad_calculada)
    # print('valor', palabra_calculada_rx[indice_error], 'en la posición: ', list_error, f'({indice_error})')
    # Arrgelar la palabra en la posicón donde está el error
    palabra_original_rx.pop(7)
    palabra_original_rx.pop(3)
    palabra_original_rx.pop(1)
    palabra_original_rx.pop(0)

    palabra_arreglada_str = ''.join(palabra_original_rx)

    # Letra donde está el error
    letra_error = df_codificacion['Letra'][df_codificacion['Fill 0s'] == palabra_arreglada_str].values[0]
    print('Letra: {}'.format(letra_error))
    print('El bit incorrecto es: {}, que está en el índice: {}'.format(palabra_calculada_rx[indice_error], indice_error))

  else:
    print('\n', '═'*100)
    # print('valor', palabra_calculada_rx[indice_error], 'en la posición: ', list_error, f'({indice_error})')
    # Arrgelar la palabra en la posicón donde está el error
    palabra_original_rx.pop(7)
    palabra_original_rx.pop(3)
    palabra_original_rx.pop(1)
    palabra_original_rx.pop(0)

    palabra_arreglada_str = ''.join(palabra_original_rx)

    # Letra donde está el error
    letra_sin_error = df_codificacion['Letra'][df_codificacion['Fill 0s'] == palabra_arreglada_str].values[0]
    print('No hay error en la letra: {}'.format(letra_sin_error))

  temp_letra_inferior += 8
  temp_letra_superior += 8


 ════════════════════════════════════════════════════════════════════════════════════════════════════
Hay un error en la siguiente letra:

Palabra original: ['0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0'] 
Palabra calculada: ['1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0']
Paridad original ['0', '1', '0', '0'] 
Paridad calculada ['1', '1', '0', '1']
Letra: d
El bit incorrecto es: 0, que está en el índice: 8

 ════════════════════════════════════════════════════════════════════════════════════════════════════
No hay error en la letra: e

 ════════════════════════════════════════════════════════════════════════════════════════════════════
No hay error en la letra: s

 ════════════════════════════════════════════════════════════════════════════════════════════════════
No hay error en la letra: l

 ════════════════════════════════════════════════════════════════════════════════════════════════════
No hay error en la letra: i

 ═════════════════════════════════════